In [2]:
import os
import torch
import pickle

from PriMeLU import PriMeLU
from options import config
from model_training import training
from data_generation import generate
from evidence_candidate import selection
from torch.nn import functional as F
import numpy as np
from tqdm import tqdm

# importing the required module
import matplotlib.pyplot as plt

## Data Generation

In [ ]:
master_path= "./ml"
if not os.path.exists("{}/".format(master_path)):
    os.mkdir("{}/".format(master_path))
    # preparing dataset. It needs about 22GB of your hard disk space.
    generate(master_path)

## Model Training

In [ ]:
# training model.
primelu = PriMeLU(config)
model_filename = "{}/models.pkl".format(master_path)
if not os.path.exists(model_filename):
    # Load training dataset.
    training_set_size = int(len(os.listdir("{}/warm_state".format(master_path))) / 4)
    supp_xs_s = []
    supp_ys_s = []
    query_xs_s = []
    query_ys_s = []
    for idx in tqdm(range(training_set_size)):
        supp_xs_s.append(pickle.load(open("{}/warm_state/supp_x_{}.pkl".format(master_path, idx), "rb")))
        supp_ys_s.append(pickle.load(open("{}/warm_state/supp_y_{}.pkl".format(master_path, idx), "rb")))
        query_xs_s.append(pickle.load(open("{}/warm_state/query_x_{}.pkl".format(master_path, idx), "rb")))
        query_ys_s.append(pickle.load(open("{}/warm_state/query_y_{}.pkl".format(master_path, idx), "rb")))
    total_dataset = list(zip(supp_xs_s, supp_ys_s, query_xs_s, query_ys_s))
    del(supp_xs_s, supp_ys_s, query_xs_s, query_ys_s)
    training(primelu, total_dataset, batch_size=config['batch_size'], num_epoch=config['num_epoch'], model_save=True, model_filename=model_filename)

## Model Testing For Warm State

In [ ]:
training_set_size = int(len(os.listdir("{}/warm_state".format(master_path))) / 4)
supp_xs_s = []
supp_ys_s = []
query_xs_s = []
query_ys_s = []
for idx in range(training_set_size):
    supp_xs_s.append(pickle.load(open("{}/warm_state/supp_x_{}.pkl".format(master_path, idx), "rb")))
    supp_ys_s.append(pickle.load(open("{}/warm_state/supp_y_{}.pkl".format(master_path, idx), "rb")))
    query_xs_s.append(pickle.load(open("{}/warm_state/query_x_{}.pkl".format(master_path, idx), "rb")))
    query_ys_s.append(pickle.load(open("{}/warm_state/query_y_{}.pkl".format(master_path, idx), "rb")))
trained_state_dict = torch.load(model_filename)
primelu.load_state_dict(trained_state_dict)
final_loss_mae = []
for i in range(training_set_size):
    prediction = primelu.forward(supp_xs_s[i].cuda(),supp_ys_s[i].cuda(),query_xs_s[i].cuda(),5)
    loss1 = F.l1_loss(prediction,query_ys_s[i].cuda().view(-1,1))
    final_loss_mae.append(loss1.item())
warm_state_test_loss = np.mean(final_loss_mae)